In [22]:
from datetime import datetime
from datetime import timezone
import polars as pl
import os

from utils import plot_sensor_measurement

start_date = datetime(2023, 10, 9, 0, 0, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2023, 10, 16, 23, 59, 59).replace(tzinfo=timezone.utc)

sensor_id = 7
filter = '1h'

data_directory = "/Users/patrickaigner/Documents/PROJECTS/ACROPOLIS/Software/acropolis-visualisation/data"
picarro_path = r"/Users/patrickaigner/Documents/PROJECTS/ACROPOLIS/Database/PICARRO"

#Calibration
# after 23.10
picarro_slope = 1.006374633215469
picarro_intercept = 0.0709482571842841

acropolis_slope = 1.015105724343444 
acropolis_intercept = -11.400846281936936

In [23]:
df_a = pl.scan_parquet(os.path.join(data_directory, "pivot_measurements.parquet")) \
    .filter(pl.col("system_name") == f"tum-esm-midcost-raspi-{sensor_id}" ) \
    .sort("creation_timestamp").filter(pl.col("creation_timestamp") \
    .is_between(start_date, end_date)) \
    .filter(pl.col("gmp343_filtered")>0.0) \
    .with_columns(((pl.col("gmp343_filtered")) * acropolis_slope + acropolis_intercept).alias("CO2_corr")) \
    .groupby_dynamic("creation_timestamp", every=filter) \
    .agg(pl.all().exclude(["creation_timestamp","DATE","TIME","datetime"]).mean())  \
    .with_columns(pl.lit("ACROPOLIS").alias("system_name"))

In [24]:
df_p = pl.scan_parquet(os.path.join(picarro_path, "picarro.parquet")) \
    .with_columns(pl.col("datetime").dt.cast_time_unit("ns").dt.replace_time_zone("UTC").alias("creation_timestamp")) \
    .sort("creation_timestamp").filter(pl.col("creation_timestamp").is_between(start_date, end_date)) \
    .with_columns((pl.col("CO2_dry") * picarro_slope + picarro_intercept).alias("CO2_corr")) \
    .groupby_dynamic("creation_timestamp", every=filter).agg(
                pl.all().exclude(["creation_timestamp","DATE","TIME","datetime"]).mean())  \
    .with_columns(pl.lit("PICARRO").alias("system_name"))

In [25]:
col_name = "CO2_corr"

df_all = pl.concat([df_a.select("creation_timestamp","system_name",col_name),df_p.select("creation_timestamp","system_name",col_name)])

plot_sensor_measurement(df_all.collect(), col_name="CO2_corr")


In [26]:
df_a = pl.scan_parquet(os.path.join(data_directory, "pivot_measurements.parquet")) \
    .filter(pl.col("system_name") == f"tum-esm-midcost-raspi-{sensor_id}" ) \
    .sort("creation_timestamp").filter(pl.col("creation_timestamp") \
    .is_between(start_date, end_date)) \
    .with_columns(((pl.col("gmp343_filtered")) * acropolis_slope + acropolis_intercept).alias("CO2_corr")) \
    .groupby_dynamic("creation_timestamp", every=filter) \
    .agg(pl.all().exclude(["creation_timestamp","DATE","TIME","datetime"]).mean())  \
    .with_columns(pl.lit("ACROPOLIS").alias("system_name"))

In [27]:
# Pressure Delta
col_name = "pressure: flow - enclosure"

df_temp = df_a.with_columns((pl.col("bme280_pressure") - pl.col("enclosure_bme280_pressure")).alias(col_name)) \
.select("creation_timestamp","system_name",col_name)

plot_sensor_measurement(df_temp.collect(), col_name=col_name)

In [28]:
# Humidity Delta
col_name = "humidity: flow - enclosure"

df_temp = df_a.with_columns((pl.col("sht45_humidity") - pl.col("enclosure_bme280_humidity")).alias(col_name)) \
.select("creation_timestamp","system_name",col_name)

plot_sensor_measurement(df_temp.collect(), col_name=col_name)

In [29]:
# Temperature Delta
col_name = "temperature: flow - enclosure"

df_temp = df_a.with_columns((pl.col("sht45_temperature") - pl.col("enclosure_bme280_temperature")).alias(col_name)) \
.select("creation_timestamp","system_name",col_name)

plot_sensor_measurement(df_temp.collect(), col_name=col_name)

# Scatter Plot, RSME, R^2